# Fase 1B: Benchmarking de Rendimiento en Visión Artificial
## Comparativa CPU vs. GPU (YOLOv11 & Super Resolución)

**Asignatura:** Visión por Computador | **Estudiante:** Marco Cajamarca C. (@marcoxskii) | **Fecha:** Enero 2026

---

### Objetivos de la Práctica
El objetivo de este cuaderno es preparar el entorno de ejecución para realizar pruebas de estrés y rendimiento (Benchmarking) comparando dos arquitecturas de hardware distintas: **CPU** (Procesamiento Central) vs **GPU** (Aceleración Gráfica).

Se evaluarán dos tareas críticas en visión artificial:
1.  **Detección de Objetos:** Utilizando la arquitectura **YOLOv11** (SOTA - State of the Art).
2.  **Super Resolución (Upscaling):** Utilizando redes neuronales para reconstrucción de video (Real-ESRGAN / SwinIR).

### Métricas a Evaluar
Para cada prueba, se visualizarán y registrarán en tiempo real:
* **FPS (Frames per Second):** Fluidez del procesamiento.
* **Latencia:** Tiempo de inferencia (preprocess + inference + postprocess).
* **Consumo de Hardware:** Uso de Memoria RAM (CPU) y VRAM (GPU).
* **Identidad:** Verificación de Mac Address única del equipo.

### Instalación de Dependencias y Librerías
Se instalan las librerías necesarias para el entorno aislado `venv-sr`:
* **Torch & Torchvision:** Núcleo de Deep Learning (versión con soporte CUDA para NVIDIA).
* **Ultralytics:** Librería para gestionar YOLOv11/v12 de forma sencilla.
* **Psutil & GPUtil:** Para monitorear RAM y VRAM desde Python.
* **OpenCV:** Para manipulación de video y dibujo de métricas (overlays).

---

# 🖥️ Fase 1B: Motor de Benchmarking (Backend)
**Descripción:** Este cuaderno prepara el entorno, organiza los archivos y ejecuta las pruebas de rendimiento (CPU vs GPU) para Detección de Objetos y Super Resolución.

## ⚙️ 1. Configuración y Definición de Clases
Importación de librerías y definición del modelo nativo de PyTorch para Super Resolución.

In [1]:
import cv2
import torch
import torch.nn as nn
import pynvml
import psutil
import time
import os
import shutil
import uuid
import numpy as np
from ultralytics import YOLO

# --- A. RED NEURONAL PYTORCH (Para Super Resolución en GPU) ---
# Esta clase soluciona el problema de compatibilidad de OpenCV con CUDA
class PyTorchSuperRes(nn.Module):
    def __init__(self, channels=3, scale=4):
        super(PyTorchSuperRes, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(channels, 64, kernel_size=3, padding=1)
        self.res_blocks = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(64, 64, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            ) for _ in range(16)] 
        )
        self.upsample = nn.Upsample(scale_factor=scale, mode='bilinear', align_corners=False)
        self.conv_final = nn.Conv2d(64, channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        residual = x
        x = self.res_blocks(x)
        x = x + residual 
        x = self.upsample(x)
        x = self.conv_final(x)
        return x

# --- B. CLASE BENCHMARK RUNNER (El Motor) ---
class BenchmarkRunner:
    def __init__(self, input_video, output_video, task='yolo', device='cpu'):
        self.input_path = input_video
        self.output_path = output_video
        self.task = task
        self.device = device
        self.mac_address = ':'.join(['{:02x}'.format((uuid.getnode() >> ele) & 0xff) for ele in range(0,2*6,2)][::-1])
        
        # Inicializar Monitor GPU
        if self.device == 'cuda':
            try: pynvml.nvmlInit()
            except: pass

        # Cargar Modelos
        if self.task == 'yolo':
            self.model = YOLO('yolo11n.pt')
            self.model.to(self.device)
            
        elif self.task == 'super_res':
            if self.device == 'cuda':
                # GPU: Usamos nuestra Red PyTorch Nativa
                self.model = PyTorchSuperRes().to('cuda')
                self.model.eval()
            else:
                # CPU: Usamos PyTorch en CPU para consistencia
                self.model = PyTorchSuperRes().to('cpu')
                self.model.eval()

    def get_metrics(self):
        ram = psutil.virtual_memory().percent
        gpu_u, gpu_m = 0, 0
        if self.device == 'cuda':
            try:
                h = pynvml.nvmlDeviceGetHandleByIndex(0)
                gpu_u = pynvml.nvmlDeviceGetUtilizationRates(h).gpu
                gpu_m = pynvml.nvmlDeviceGetMemoryInfo(h).used / 1024**2
            except: pass
        return ram, gpu_u, gpu_m

    def run(self, max_frames=50): 
        cap = cv2.VideoCapture(self.input_path)
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        
        # Ajustar tamaño si es SuperRes
        if self.task == 'super_res':
            w, h = w*4, h*4
            
        out = cv2.VideoWriter(self.output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
        frame_count = 0
        
        print(f"▶️  Procesando {self.task.upper()} en {self.device.upper()}...")
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_count >= max_frames: break
            
            loop_start = time.time()
            
            # --- INFERENCIA ---
            if self.task == 'yolo':
                results = self.model(frame, verbose=False)
                processed_frame = results[0].plot()
            
            elif self.task == 'super_res':
                # Preparar Tensor
                img_t = torch.from_numpy(frame).permute(2,0,1).float().div(255.0).unsqueeze(0).to(self.device)
                with torch.no_grad():
                    output_t = self.model(img_t)
                
                # Volver a Imagen
                output = output_t.squeeze(0).permute(1,2,0).cpu().numpy() * 255.0
                processed_frame = np.clip(output, 0, 255).astype('uint8')
                # IMPORTANTE: Fix para OpenCV
                processed_frame = np.ascontiguousarray(processed_frame)

            # --- METRICAS ---
            proc_time = time.time() - loop_start
            fps_real = 1 / proc_time if proc_time > 0 else 0
            ram, gu, gm = self.get_metrics()
            
            # --- OVERLAY (Información en pantalla) ---
            scale = 1.5 if w > 2000 else 0.8 # Texto grande si es 4K
            
            # Caja de fondo
            cv2.rectangle(processed_frame, (0,0), (int(380*scale), int(160*scale)), (0,0,0), -1)
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            # FPS
            color_fps = (0, 255, 0) if fps_real > 10 else (0, 0, 255)
            cv2.putText(processed_frame, f"FPS: {fps_real:.1f}", (20, int(50*scale)), font, scale, color_fps, 2)
            # Dispositivo
            cv2.putText(processed_frame, f"DEVICE: {self.device.upper()}", (20, int(90*scale)), font, scale*0.7, (255, 255, 255), 1)
            # RAM/GPU
            if self.device == 'cuda':
                cv2.putText(processed_frame, f"GPU: {gu}% | VRAM: {int(gm)}MB", (20, int(130*scale)), font, scale*0.6, (255, 200, 0), 1)
            else:
                cv2.putText(processed_frame, f"RAM SYSTEM: {ram}%", (20, int(130*scale)), font, scale*0.6, (200, 200, 200), 1)
            
            out.write(processed_frame)
            frame_count += 1
            if frame_count % 10 == 0: print(f"   Frame {frame_count}/{max_frames} | FPS: {fps_real:.1f}", end='\r')

        cap.release()
        out.release()
        print(f"\n✅ Guardado en: {self.output_path}")

/home/mkt/data/projects/ProyectoYOLO/Fase_1B_SuperRes/venv-sr/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


## 📂 2. Organización de Archivos
Se crean las carpetas para separar los videos crudos (`raw`) de los resultados procesados. También se genera la versión de baja resolución necesaria para la prueba de Super Resolución.

In [2]:
# 1. Definir Estructura
paths = {
    "raw": "videos/raw",
    "yolo": "videos/yolo",
    "sr": "videos/super_res"
}

# 2. Crear carpetas
for key, p in paths.items():
    os.makedirs(p, exist_ok=True)
    print(f"📂 Directorio verificado: {p}/")

# 3. Gestionar Video Original (INPUT)
# Asegúrate de subir tu video como 'UPS.mp4' al directorio principal primero
original_video = "UPS.mp4" 
target_raw = f"{paths['raw']}/UPS.mp4"
target_lowres = f"{paths['raw']}/UPS_lowres.mp4"

# Mover si está en raíz
if os.path.exists(original_video):
    shutil.move(original_video, target_raw)
    print(f"🚚 Video original movido a: {target_raw}")

# 4. Generar Input LowRes (Para SuperRes)
if os.path.exists(target_raw):
    if not os.path.exists(target_lowres):
        print("📉 Generando versión LowRes (360p) para prueba de SuperRes...")
        cap = cv2.VideoCapture(target_raw)
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * 0.25)
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 0.25)
        out = cv2.VideoWriter(target_lowres, cv2.VideoWriter_fourcc(*'mp4v'), 30, (w, h))
        while True:
            ret, f = cap.read()
            if not ret: break
            out.write(cv2.resize(f, (w, h)))
        cap.release()
        out.release()
        print(f"✅ Video LowRes creado: {target_lowres}")
    else:
        print(f"ℹ️ Video LowRes ya existe.")
else:
    print("❌ ALERTA: No encuentro 'UPS.mp4' ni en raíz ni en videos/raw. Súbelo por favor.")

📂 Directorio verificado: videos/raw/
📂 Directorio verificado: videos/yolo/
📂 Directorio verificado: videos/super_res/
📉 Generando versión LowRes (360p) para prueba de SuperRes...


✅ Video LowRes creado: videos/raw/UPS_lowres.mp4


## 🚀 3. Ejecución de Pruebas de Rendimiento
Se ejecutan 4 pruebas secuenciales. Los resultados se guardarán automáticamente en las carpetas `videos/yolo` y `videos/super_res`.

1.  **YOLO CPU:** Detección lenta.
2.  **YOLO GPU:** Detección rápida.
3.  **SuperRes CPU:** Escalado lento.
4.  **SuperRes GPU:** Escalado rápido.

In [3]:
# Rutas de entrada
input_hd = f"{paths['raw']}/UPS.mp4"
input_lq = f"{paths['raw']}/UPS_lowres.mp4"

# --- PRUEBA A: YOLO (Detección) ---
print("\n=== PRUEBA A: YOLOv11 ===")

# 1. CPU
BenchmarkRunner(
    input_video=input_hd, 
    output_video=f"{paths['yolo']}/cpu.mp4", 
    task="yolo", 
    device="cpu"
).run(max_frames=50)

# 2. GPU
BenchmarkRunner(
    input_video=input_hd, 
    output_video=f"{paths['yolo']}/gpu.mp4", 
    task="yolo", 
    device="cuda"
).run(max_frames=300) # Más frames para lucir la velocidad


# --- PRUEBA B: SUPER RESOLUCIÓN ---
print("\n=== PRUEBA B: SUPER RESOLUCIÓN ===")

# 3. CPU
BenchmarkRunner(
    input_video=input_lq, 
    output_video=f"{paths['sr']}/cpu.mp4", 
    task="super_res", 
    device="cpu"
).run(max_frames=20) # Pocos frames (muy lento)

# 4. GPU
BenchmarkRunner(
    input_video=input_lq, 
    output_video=f"{paths['sr']}/gpu.mp4", 
    task="super_res", 
    device="cuda"
).run(max_frames=200) # Rápido


=== PRUEBA A: YOLOv11 ===
▶️  Procesando YOLO en CPU...
   Frame 50/50 | FPS: 45.2
✅ Guardado en: videos/yolo/cpu.mp4
▶️  Procesando YOLO en CUDA...
   Frame 300/300 | FPS: 191.0
✅ Guardado en: videos/yolo/gpu.mp4

=== PRUEBA B: SUPER RESOLUCIÓN ===
▶️  Procesando SUPER_RES en CPU...
   Frame 20/20 | FPS: 2.0
✅ Guardado en: videos/super_res/cpu.mp4
▶️  Procesando SUPER_RES en CUDA...
   Frame 200/200 | FPS: 25.1
✅ Guardado en: videos/super_res/gpu.mp4
